In [1]:
!pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="<API_KEY>"
os.environ["LANGCHAIN_PROJECT"]="RAG_ADVANCED"
os.environ['OPENAI_API_KEY'] = "<API_KEY>"


from langsmith import utils
utils.tracing_is_enabled()

True

In [2]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [4]:
#Load Documents
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/", ),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [5]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

#Embed
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

vectorestore = Chroma.from_documents(documents=splits, embedding = embedding, persist_directory="./db006")

#Retriever
retriever = vectorestore.as_retriever()

In [7]:
#Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input":"Could the members of the Police perform lawful arrests?",
        "output":"What can the members of the Police do?",
    },
    {
        "input":"Jan Sindel's was born in what country?",
        "output":"what is Jan Sindel's personal history?",
    },
]
#We now transform these example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
             "system",
             """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        #Few shot examples
        few_shot_prompt,
        #New question
        ("human", "{question}"),
    ]
)

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

generate_queries_step_back = prompt | llm | StrOutputParser()

question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question":question})

'What does task decomposition mean in the context of agents?'

In [11]:
#Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be ccomprehensive and not contradicted with the following context
if they are relevant. Otherwise, ignore them if they are not relevant.
#{normal_context}
#{step_back_context}

#Original Question:{question}
#Answer:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        #Retrieve context using the normal question
        "normal_context":RunnableLambda(lambda x:x["question"]) | retriever,
        #Retrieve context using the step-back question
        "step_back_context":generate_queries_step_back | retriever,
        #Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)


chain.invoke({"question":question})

'Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals. This approach enables the efficient handling of intricate tasks by allowing the agent to focus on individual components rather than attempting to tackle the entire task at once.\n\nThere are several methods for achieving task decomposition:\n\n1. **Prompting**: LLMs can be prompted with simple questions or statements, such as "Steps for XYZ." or "What are the subgoals for achieving XYZ?" This method leverages the model\'s ability to generate structured responses based on the input it receives.\n\n2. **Task-Specific Instructions**: Providing specific instructions tailored to the task can guide the LLM in breaking down the task effectively. For example, asking the model to "Write a story outline" for a novel gives it a clear framework to work within.\n\n3. **Human Inputs**: Human guidance can also play a crucial role in task decomposition.